<a href="https://colab.research.google.com/github/vartikagpt10/Sound-Event-Detection---ML-in-Signal-Processing/blob/main/MLSP_Project_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import librosa
import pdb
import string
import os

!pip install python-levenshtein

from Levenshtein import distance

In [ ]:
def wav2feat(wavfile):
    '''
    Input: audio wav file name
    Output: Magnitude spectrogram
    '''
    x, Fs = librosa.load(wavfile, sr=44100, mono=True) 
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X)

def wavs2feat(wavfiles):
    '''
    Concatenate the audio files listed in wavfiles
    Input: list of audio wav file names
    Output: Magnitude spectrogram of concatenated wav
    '''
    x = []
    for wf in wavfiles:
        x1, Fs = librosa.load(wf, sr=44100, mono=True)
        x.append(x1)
    x = np.hstack(x)
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X)

def read_csv(filename):
    id_label = {}
    with open(filename,'r') as fid:
        for line in fid: # '176787-5-0-27.wav,engine_idling\n'
            tokens = line.strip().split(',') # ['176787-5-0-27.wav', 'engine_idling']
            id_label[tokens[0]] = tokens[1]
    return id_label

def editDistance(gt, est):
    '''both are lists of labels
    E.g. gt is "dog_bark-street_music-engine_idling"
    E.g. est is "street_music-engine_idling"
    '''
    gttokens = gt.split('-')
    esttokens = est.split('-')
    # Map token to char
    tokenset = list(set(gttokens+esttokens)) # ['dog_bark', 'siren', 'street_music', 'engine_idling']
    token_char = {}
    for i in range(len(tokenset)):
        token_char[tokenset[i]] = string.ascii_uppercase[i]  # {'dog_bark': 'A', 'siren': 'B', 'street_music': 'C', 'engine_idling': 'D'}
    # convert gt and est to strings
    gtstr = [token_char[t] for t in gttokens]
    gtstr = ''.join(gtstr)  # 'BCA'
    eststr = [token_char[t] for t in esttokens]
    eststr = ''.join(eststr)  # 
    # Compare
    editdist = distance(gtstr, eststr) # 1
    score = 1 - editdist/len(gtstr)
    return editdist, score

def evals(gtcsv, estcsv, taskid):
    gt_id_label = read_csv(gtcsv)
    est_id_label = read_csv(estcsv)
    score = 0
    for id in est_id_label:
        if taskid==1:
            if est_id_label[id] == gt_id_label[id]:
                score += 1
        elif taskid==2:
            _, ss = editDistance(gt_id_label[id], est_id_label[id])
            score += ss
        else:
            pdb.set_trace()
            assert False, ["taskid not correct; it is", taskid]
    avgScore = score/len(est_id_label)
    return avgScore

# if __name__=="__main__":
    # wavs = ['../shared_train/audio_train/180937-7-3-27.wav']
    # wavs = ['/content/drive/MyDrive/Training data/audio_train/100652-3-0-0.wav']
    # print(wavs2feat(wavs).shape)
    # wavfiles = ['../shared_train/audio_train/180937-7-3-27.wav','../shared_train/audio_train/180937-7-3-27.wav']
    # X = wavs2feat(wavs)
    # eval('test_task1/labels.csv', 'test_task1/est.csv', 1)
    # editDistance("dog_bark-street_music-engine_idling","siren-street_music-engine_idling")

#**TASK1**





##Data Extraction

In [ ]:
root = '/content/drive/MyDrive/Project/MLSP Course Project/audio_train_1ch/'
df = pd.read_csv('/content/drive/MyDrive/Project/MLSP Course Project/labels_train.csv')
#root = '/content/drive/MyDrive/Training data/audio_train_1ch/'
#df = pd.read_csv('/content/drive/MyDrive/Training data/labels_train.csv')
   
i = 0 
hyphen = '-'
train_subarray0, train_subarray1, train_subarray2, train_subarray3, train_subarray4, train_subarray5, train_subarray6, train_subarray7, train_subarray8, train_subarray9 = [],[],[],[],[],[],[],[],[],[]
train_subarray = [train_subarray0, train_subarray1, train_subarray2, train_subarray3, 
                  train_subarray4, train_subarray5, train_subarray6, train_subarray7, train_subarray8, train_subarray9] #stores the spectrograms of each class separately
for file in df.slice_file_name:
  j = df['slice_file_name'][i][df['slice_file_name'][i].find(hyphen) +1] # dataset separated into its classes using the file name 
  j = int(j)
  if file.endswith(".wav"):
    sample = wav2feat(root + file) 
    concan = np.zeros((513,401-sample.shape[1])) #adding zeros to make all the entries of the spectogram of the same dimension
    sample = np.concatenate((sample, concan), axis=1)
    print(i) #to keep track of number of files extracted
    i = i+1
    train_subarray[j].append(sample)
train_subarray = np.array(train_subarray)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## Data Set Creation

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
train_array, val_array = [[]], [[]]
for i in range(10):
  len_to_split = len(train_subarray[i]) 
  len_train = int(0.8 * len_to_split)
  for j in range(len_to_split):
    tup = [train_subarray[i][j],i]
    if j < len_train:
      train_array.append(tup)
    else:
      val_array.append(tup)

train_array = np.array(train_array)
val_array = np.array(val_array)

#Remove the empty entry from each array
train_array = np.delete(train_array,0)
val_array = np.delete(val_array,0)

In [ ]:
train_array = shuffle(train_array)
val_array = shuffle(val_array)

X_data = [] #stores the training spectrograms
Y_data = []
for i in range(len(train_array)):
  X_data.append(train_array[i][0])
  Y_data.append(train_array[i][1])
X_data = np.array(X_data)
Y_data = np.hstack(Y_data)
Y_data = Y_data.reshape(Y_data.size,1)

X_data1 = [] #stores the validation spectrograms
Y_data1 = []
for i in range(len(val_array)):
  X_data1.append(val_array[i][0])
  Y_data1.append(val_array[i][1])
X_data1 = np.array(X_data1)
Y_data1 = np.hstack(Y_data1)
Y_data1 = Y_data1.reshape(Y_data1.size,1)

In [ ]:
X_train = X_data.reshape(X_data.shape[0], 513, 401,1)
X_val = X_data1.reshape(X_data1.shape[0], 513, 401,1)

In [ ]:
#one hot encoding
Y_train = to_categorical(Y_data)
Y_val = to_categorical(Y_data1)

##Training

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Dropout, Dense, MaxPooling2D, GlobalAveragePooling2D, LayerNormalization, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras import losses

In [ ]:
#CNN with 7 layers
def model1():
  inputs = keras.Input(shape = (513,401,1))
  model1 = tf.keras.Sequential()
  model1.add(LayerNormalization(axis=2))
  model1.add(Conv2D(filters=16, kernel_size=3, input_shape=(513, 401, 1), activation='relu'))
  model1.add(MaxPooling2D(pool_size=2))
  model1.add(Dropout(0.2))

  model1.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
  model1.add(MaxPooling2D(pool_size=2))
  model1.add(Dropout(0.2))

  model1.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
  model1.add(MaxPooling2D(pool_size=2))
  model1.add(Dropout(0.2))

  model1.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
  model1.add(MaxPooling2D(pool_size=2))
  model1.add(Dropout(0.2))
  model1.add(GlobalAveragePooling2D())

  model1.add(Dense(10, activation='softmax'))
  model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model1

In [ ]:
#CNN with 4 layers
def model2():
  inputs = keras.Input(shape = (513,401,1))
  x = LayerNormalization(axis = 2)(inputs)
  x = Conv2D(8, kernel_size=(7,7), activation='relu', padding='same')(x)
  x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
  x = Conv2D(16, kernel_size=(5,5), activation='relu', padding='same')(x)
  x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
  x = Conv2D(16, kernel_size=(3,3), activation='relu', padding='same')(x)
  x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
  x = Conv2D(32, kernel_size=(3,3), activation='relu', padding='same')(x)
  x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
  x = Conv2D(32, kernel_size=(3,3), activation='relu', padding='same')(x)
  x = Flatten()(x)
  x = Dropout(0.2)(x)
  x = Dense(64, activation='relu', activity_regularizer=tf.keras.regularizers.l2(0.001))(x)
  o = Dense(10, activation='softmax')(x)
  model = tf.keras.Model(inputs=inputs, outputs=o)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model  

In [ ]:
###DNN
from keras.models import Sequential
from keras.layers import Dense
def model4():
  inputs = keras.Input(shape = (513,401,1))
  model4 = Sequential()
  model4.add(Flatten())
  model4.add(Dropout(0.2))
  model4.add(Dense(512, input_shape=(X_train.shape[0],513,401,1), activation='relu'))
  model4.add(Dense(256, activation='relu'))
  model4.add(Dense(128, activation='relu'))
  model4.add(Dense(10, activation='softmax'))
  model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model4

In [ ]:
'''opt = optimizers.Adam()
model1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model1.fit(x=X_train, y=Y_train, validation_split = 0.2, verbose = 2, epochs = 150, shuffle = True, batch_size = 32)'''

In [ ]:
#model4.fit(X_train, Y_train, epochs=50, verbose=1)
#loss, accuracy= model4.evaluate(X_val, Y_val, verbose=1)

In [ ]:
#For using model 1
model = model1()
history = model.fit(x=X_train, y=Y_train, verbose = 2, epochs = 150, shuffle = True, batch_size = 32)

Epoch 1/150
44/44 - 4s - loss: 2.1251 - accuracy: 0.1649
Epoch 2/150
44/44 - 3s - loss: 1.8165 - accuracy: 0.3504
Epoch 3/150
44/44 - 3s - loss: 1.5909 - accuracy: 0.4499
Epoch 4/150
44/44 - 3s - loss: 1.3758 - accuracy: 0.5203
Epoch 5/150
44/44 - 3s - loss: 1.2635 - accuracy: 0.5529
Epoch 6/150
44/44 - 3s - loss: 1.1187 - accuracy: 0.6162
Epoch 7/150
44/44 - 3s - loss: 1.0257 - accuracy: 0.6326
Epoch 8/150
44/44 - 3s - loss: 1.0000 - accuracy: 0.6539
Epoch 9/150
44/44 - 3s - loss: 0.9350 - accuracy: 0.6674
Epoch 10/150
44/44 - 3s - loss: 0.8892 - accuracy: 0.6866
Epoch 11/150
44/44 - 3s - loss: 0.8682 - accuracy: 0.7086
Epoch 12/150
44/44 - 3s - loss: 0.8194 - accuracy: 0.7228
Epoch 13/150
44/44 - 3s - loss: 0.7985 - accuracy: 0.7349
Epoch 14/150
44/44 - 3s - loss: 0.7401 - accuracy: 0.7520
Epoch 15/150
44/44 - 3s - loss: 0.7342 - accuracy: 0.7584
Epoch 16/150
44/44 - 3s - loss: 0.7285 - accuracy: 0.7584
Epoch 17/150
44/44 - 3s - loss: 0.6923 - accuracy: 0.7669
Epoch 18/150
44/44 - 3s

In [ ]:
model.evaluate(X_val, Y_val)

12/12 [==============================] - 0s 27ms/step - loss: 2.4300 - accuracy: 0.6751


[2.4299561977386475, 0.6751412153244019]

##Voting on Different Models

In [ ]:
from sklearn.ensemble import VotingClassifier 
from sklearn.metrics import accuracy_score 

# group / ensemble of models 
estimator = [] 
estimator.append(('CNN_2', model2())) 
estimator.append(('DNN', model4())) 
estimator.append(('CNN_1', model1())) 
  
# Voting Classifier with hard voting 
vot_hard = VotingClassifier(estimators = estimator, voting ='hard') 
vot_hard.fit(X_train, Y_train)
vot_hard.evaluate(X_val, Y_val) 
Y_pred = vot_hard.predict(X_test) 
model = vot_hard

print("Hard Voting Score % d" % score)

##Sample Test Data

In [ ]:
root1 = '/content/drive/MyDrive/Project/MLSP Course Project/sample_test_task1/feats/'
df1 = pd.read_csv('/content/drive/MyDrive/Project/MLSP Course Project/sample_test_task1/labels.csv')
#root1 = '/content/drive/MyDrive/Training data/sample_test_task1/feats/'
#df1 = pd.read_csv('/content/drive/MyDrive/Training data/sample_test_task1/labels.csv')
df1 = np.array(df1)
test_data = []
sample = np.load(root1 + 'a00001.npy') 
concan = np.zeros((513,401-sample.shape[1]))
sample = np.concatenate((sample, concan), axis=1)
test_data.append(sample)
i = 1
for file in df1[:,:1]:
  sample = np.load(root1 + file[0] + '.npy') 
  concan = np.zeros((513,401-sample.shape[1]))
  sample = np.concatenate((sample, concan), axis=1)
  test_data.append(sample)
  print(i)
  i = i+1
test_data = np.array(test_data)
X_test = test_data.reshape(test_data.shape[0], 513, 401, 1)

In [ ]:
pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)

enc = {'air_conditioner': 0, 'car_horn': 1, 'children_playing': 2, 'dog_bark': 3, 'drilling': 4, 'engine_idling': 5, 'gun_shot': 6, 'jackhammer': 7, 'siren': 8, 'street_music': 9}
cl = list(enc.keys())
la = list(enc.values())
 
Y_pred = []         #list of predicted labels
for i in range(len(pred)):
  position = la.index(pred[i])
  Y_pred.append(cl[position])

##Test Data

In [ ]:
test_x=[]
for i in range(1,51):
  if (i<10):
    path='/content/drive/MyDrive/Project/MLSP Course Project/feats/a00'+str(i)+'.npy'
  else:
    path='/content/drive/MyDrive/Project/MLSP Course Project/feats/a0'+str(i)+'.npy'
  spect=np.load(path)
  concan = np.zeros((513,401-spect.shape[1]))
  sample = np.concatenate((spect, concan), axis=1)
  test_x.append(sample)

In [ ]:
test_x = np.array(test_x)

In [ ]:
test_x.reshape(50,513,401,1)

In [ ]:
pred = model.predict(test_x)
pred = np.argmax(pred,axis=1)

enc = {'air_conditioner': 0, 'car_horn': 1, 'children_playing': 2, 'dog_bark': 3, 'drilling': 4, 'engine_idling': 5, 'gun_shot': 6, 'jackhammer': 7, 'siren': 8, 'street_music': 9}
cl = list(enc.keys())
la = list(enc.values())
 
Y_pred = []         #list of predicted labels
for i in range(len(pred)):
  position = la.index(pred[i])
  Y_pred.append(cl[position])

##Results

In [ ]:
Y_pred

['air_conditioner',
 'street_music',
 'dog_bark',
 'jackhammer',
 'children_playing',
 'car_horn',
 'street_music',
 'gun_shot',
 'jackhammer',
 'dog_bark',
 'gun_shot',
 'air_conditioner',
 'children_playing',
 'car_horn',
 'engine_idling',
 'siren',
 'drilling',
 'children_playing',
 'drilling',
 'car_horn',
 'jackhammer',
 'street_music',
 'gun_shot',
 'engine_idling',
 'jackhammer',
 'air_conditioner',
 'air_conditioner',
 'street_music',
 'children_playing',
 'children_playing',
 'car_horn',
 'street_music',
 'siren',
 'engine_idling',
 'children_playing',
 'drilling',
 'dog_bark',
 'gun_shot',
 'engine_idling',
 'gun_shot',
 'drilling',
 'drilling',
 'drilling',
 'jackhammer',
 'dog_bark',
 'children_playing',
 'street_music',
 'drilling',
 'jackhammer',
 'drilling']

In [ ]:
fname = []
for i in range(1,51):
  if (i<10):
    temp_str='a00'+str(i)+'.npy'
  else:
    temp_str='a00'+str(i)+'.npy'
  fname.append(temp_str)
dict = {'name': fname, 'label': Y_pred}
df2 = pd.DataFrame(dict)
df2.to_csv('/content/drive/MyDrive/Project/MLSP Course Project/task1_labels_test.csv', header=False, index=False)

In [ ]:
tf.keras.backend.clear_session()